In [1]:
# Install package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install mysql-connector-python
import mysql.connector
from tqdm import tqdm

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="915997582",
  database="EDDB"  
)

print(db)

cursor = db.cursor()

In [5]:
# How many systems does each faction control
query = """
SELECT COUNT(id), allegiance
FROM system_populated
GROUP BY allegiance
ORDER BY COUNT(id) DESC
"""

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

(10507, 'Independent')
(4556, 'Empire')
(4386, 'Federation')
(1092, 'Alliance')
(19, '-1')
(9, 'Pilots Federation')


In [6]:
# Find the commodity with the largest profit margin.
query = """
SELECT name, max_sell_price, min_buy_price
FROM commodity
ORDER BY (max_sell_price - min_buy_price) DESC
LIMIT 5
"""

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

('Tritium', 52455, 4005)
('Meta-Alloys', 236737, 190461)
('Agronomic Treatment', 30998, 322)
('Ceramic Composites', 27883, 77)
('CMM Composite', 20176, 4807)


In [12]:
# Find the system that sells a given commodity for the lowest price.

name = "Beer"

# get price
query = """
SELECT min_buy_price
FROM commodity
WHERE name = "{name}"
""".format(name=name)

cursor.execute(query)

results = cursor.fetchall()

price = 0

for x in results:
    price = x[0]

# where is it
query = """
SELECT system_populated.name
FROM listing
JOIN station ON station.id=listing.station_id
JOIN system_populated ON station.system_id=system_populated.id
JOIN commodity ON listing.commodity_id=commodity.id
WHERE commodity.name = "{name}" AND listing.buy_price = {price}
""".format(name=name, price=price)

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

('LFT 37',)


In [17]:
# Find the system that buys a given commodity for the highest price.
# BROKEN
name = "Beer"

# get price
query = """
SELECT max_sell_price
FROM commodity
WHERE name = "{name}"
""".format(name=name)

cursor.execute(query)

results = cursor.fetchall()

price = 0

for x in results:
    price = x[0]

# where is it
query = """
SELECT system_populated.name
FROM listing
JOIN station ON station.id=listing.station_id
JOIN system_populated ON station.system_id=system_populated.id
JOIN commodity ON listing.commodity_id=commodity.id
WHERE commodity.name = "{name}" AND listing.buy_price = {price}
""".format(name=name, price=price)

cursor.execute(query)

results = cursor.fetchall()

for x in results:
    print(x)

(4763,)
[]
